In [ ]:
from run import run, getOutputFileName
import os 
from hydra import compose, initialize
from omegaconf import OmegaConf
import logging 
import json
import pickle

def run_all_experiments_for_attack(attack_name: str, output_dir: str, attack_ratio=0.25, defence=False):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created directory: {output_dir}")
    results_path = output_dir + attack_name + "/"
    if not os.path.exists(results_path): 
        os.makedirs(results_path)
        print(f"Created directory {results_path}")
    logging.basicConfig(filename=output_dir+"experiment_errors.log", level=logging.DEBUG, format='%(asctime)s - %(levelname)s: %(message)s')
    new_configs_path =output_dir+"runconfigs/"
    if os.path.exists(new_configs_path):
        os.rmdir(new_configs_path)
    os.makedirs(new_configs_path)

    for model_file in ["rnn"]: #"base", "mlp", "lgr", "lsvc", "xgb_bagging", "rf", "lstm", rnn
        with initialize(version_base=None, config_path="conf", job_name="test_app"):
            cfg = compose(config_name=model_file)
        num_clients_per_model = {
            "SCNN": 100, 
            "MLP": 100,
            "LGR": 100, 
            "LSVC": 100, 
            "LSTM": 50,
            "RNN": 50, 
            "RF": 50,
            "XGB": 50 
        }
        experiment_config = [("END", attack_ratio)] #("FULL", 0), , ("MID", attack_ratio), , ("END", attack_ratio)("FULL", 0), ("FULL", attack_ratio), ("MID", attack_ratio), 
        model = cfg.model
        print("Running", attack_name, "on", model)
        cfg.num_clients, cfg.num_clients_per_round_fit = num_clients_per_model[model], num_clients_per_model[model]
        cfg.attack_type=attack_name
        cfg.defence = defence
        if(defence):
            print("-----------------------------OUTLIER DETECTION ENABLED-------------------------------------")
        for (attack_round, ratio) in experiment_config: 
            cfg.max_attack_ratio = ratio 
            cfg.attack_round = attack_round
            try:
                config_file_name = f"{model}_{attack_name}_{attack_round}_{ratio*100}"
                OmegaConf.save(config=cfg, f=new_configs_path+config_file_name+".yaml") 
                results = run(config_path=new_configs_path, config_name=config_file_name)
                with open(results_path+getOutputFileName(model=model, attack_round=attack_round, attack_ratio=ratio), "wb") as h: 
                    pickle.dump(results, h, protocol=pickle.HIGHEST_PROTOCOL)
                    
            except Exception as e: 
                error_message = {"attack_name":attack_name, "model":model, "error":str(e)}
                error_message = json.dumps(error_message)
                logging.error(error_message)
                logging.log
    for handler in logging.root.handlers[:]:
        handler.close()

In [ ]:
run_all_experiments_for_attack(attack_name="LF", output_dir="runsrnn1/")

In [ ]:
run_all_experiments_for_attack(attack_name="MPAF", output_dir="runslstm3/")

In [ ]:
run_all_experiments_for_attack(attack_name="MPAF", output_dir="runslstm3/")